#### Smali & Manifest File Analysis

In [1]:
from os import walk
import pandas as pd
import os
from tqdm import tqdm

In [2]:
#   TODO Add loading bar to this lengthy function
def decompileAll():

    # 1: Traverse all APKs in the folder:
    dir = 'C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\ransomware'

    for filename in tqdm(os.listdir(dir)):
        if filename.endswith('.apk'):
            print("decompiling: "+ filename)
            
            #set output directory (removing .apk from the end)
            foldername = 'out_' + filename[:-4]
            var1 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\aples\\' + foldername
        
            #set input apk
            var2 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\ransomware\\' + filename

            #convert python vars to batch vars
            os.putenv("VAR1", var1)
            os.putenv("VAR2", var2)

            #call the run.bat for the apktool
            os.system("run.bat")

            print("3")
    print("...Done!")

In [3]:
#   Create empty lists
methods_to_find = []
perms_to_find = []

#   Read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions

df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [4]:
#   Create class object to be used in the lists.
#   Object will contain the name of the feature and how many times it's been found
class obj:
    def __init__(self, name):
        self.name = name
        self.count = 0
    def print(self):
        print(self.name + ": " + str(self.count))

In [5]:
#   Initialize empty lists
myList = []
fakeList = []

#   Fill lists with permissions and functions. 
#   The "i" refers to a term in the list, not index.
for i in perms_to_find:
    myList.append(obj(i))
    fakeList.append(i)

for i in methods_to_find:
    myList.append(obj(i))
    fakeList.append(i)

myList.insert(0, obj('activity'))
fakeList.insert(0, 'activity')

myList.insert(0, obj('service'))
fakeList.insert(0, 'service')

myList.insert(0, obj('receiver'))
fakeList.insert(0, 'receiver')

#   Append the last item (name)
myList.append('')
fakeList.append('Name')

#   Self print function for list
def printList(list):
    for i in range(len(list)):
        try:
            list[i].print()
        except AttributeError:
            print(list[i])

In [6]:
#   Create empty dataframe
df = pd.DataFrame(columns=fakeList)

#   This clears the CSV if it exists
df.to_csv('newRansom.csv', mode='w', header=True)

In [7]:
#   The analysis function
def findFunctionsInDir(direct, list=0, verbose = True, filename = ''):

    #for each directory, and files
    for dirpath, dirnames, files in walk(direct):
        
        #fill the name field in the list
        myList[fakeList.index('Name')] = filename

        #skip the res folder, too many subfolders, slows down.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:
            
            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):

                        if('<activity ' in line):
                                myList[fakeList.index('activity')].count += 1
                        if('<service ' in line):
                                myList[fakeList.index('service')].count += 1
                        if('<receiver ' in line):
                                myList[fakeList.index('receiver')].count += 1

                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #Set permission found to 1
                                myList[fakeList.index(i)].count = 1

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):

                                #Increase counter for found item
                                myList[fakeList.index(i)].count += 1

In [8]:
#function saves to csv file
def saveToFile(df):
    df.to_csv('newRansom.csv', mode='a', header=False)  

In [9]:
def appendToDF(list, df):
    #   Initialize empty list for the counts only.
    listOfCount = []
    
    #   Fill such list (except if it's the last one [Name])
    for i in range(len(list)):
        if(i < len(list)-1):
            listOfCount.append(list[i].count)
        else:
            listOfCount.append(list[i])

    #   Append the list to the dataframe
    df.loc[len(df.index)] = listOfCount

In [10]:
#   for the passed directory, do: 
#   Call find function, append results of 'myList' to the df, save.
def runAnalysis(filename, single_file):
    
    findFunctionsInDir(direct=filename, verbose=False, filename = single_file)
    appendToDF(myList, df)

    #TODO not sure why only this method works, but not backing up the list for efficiency.
    myList.clear()
    for k in perms_to_find:
        myList.append(obj(k))

    for k in methods_to_find:
        myList.append(obj(k))
    
    myList.insert(0, obj('activity'))
    myList.insert(0, obj('service'))
    myList.insert(0, obj('receiver'))
    myList.append('')

In [11]:
#decompile all of the APKs in folder (hardcoded)
#decompileAll()

In [12]:
#This line will clear the DF (call if needed)
df = df.iloc[0:0]

#APP = "wannaLocker"

direc = "C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\decompiled_ransomware\\"
lim_counter = 0
#   Limits the number of APKs
LIMIT = 2200

#LEN = len(APP) + 1

#   Here, filename means folder
#   The tqdm is the loading bar library
for filename in tqdm(os.listdir(direc)):

    if(lim_counter < LIMIT):
        try:
            #   If this line does not work, check to see if path is correct
            runAnalysis(os.path.join(direc, filename), filename)

            #   Increase lim_counter
            lim_counter += 1
        except UnicodeDecodeError:
            print("Unicode Error on file: " + filename)
        except FileNotFoundError:
            print("File Not Found on file: " + filename)

saveToFile(df)
print("..done! Saved to csv.")

100%|██████████| 1790/1790 [02:00<00:00, 14.91it/s]

..done! Saved to csv.


In [13]:
#saveToFile(df)
#print("..done! Saved to csv.")

#1653 done so far

In [14]:
df

,receiver,service,activity,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,...,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto,Name
0,4,2,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,out_00357b0e208c20df3182d54cb2ba15bf
1,2,7,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,out_0035d75d5b143a92d5b55c5063f63bd0
2,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,out_0064a447770d0045673ed2a64e129636
3,2,7,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,out_006a476515e90faeb71753a410a8a0d9
4,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,out_008aafc94cfdc5e41d8f5fa41abe23ac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,3,4,4,1,1,1,1,1,1,1,...,0,0,0,28,5,0,0,0,0,out_ff3738f7ed1348515cbc58b3d8e0d9ef
1786,2,7,5,1,1,1,1,1,1,1,...,0,0,7,0,0,0,0,0,0,out_ff6f63405f967c64efb786ccbf9bd642
1787,1,0,1,1,1,1,1,1,1,1,...,0,0,7,0,5,0,0,0,0,out_ff97a8297a2607f2e7418f7b1f344ba4
1788,2,6,5,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,out_ffaa4f2991c54a35329b43b2de31cc8c


In [15]:
#This line will clear the DF (call if needed)
#df = df.iloc[0:0]

#TODO: Add a way to stop/resume analysis